In [13]:
from pynput.mouse import Controller, Button
import cv2
import time
import numpy as np
import handtrackingmodule as htm

# Initialize camera and hand detector
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Frame width
cap.set(4, 720)   # Frame height

detector = htm.handDetector(detectionCon=0.7)
mouse = Controller()
dragging = False  # Dragging state
last_click_time = 0
click_cooldown = 0.5  # Cooldown for better response
drag_cooldown = 0.5  # Prevents instant re-drag

# Screen & padding settings
screen_w, screen_h = 1920, 1080  # Change according to your screen
frame_w, frame_h = 1280, 720     # Webcam frame size
padding = 200  # Reduce hand movement needed
last_drag_time = 0  # Track last drag action

# Cursor smoothing parameters
smooth_factor = 0.2  # Controls smoothness (higher = smoother)
prev_x, prev_y = 0, 0  # Store previous cursor positions

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Mirror image
    img = detector.findHands(img)
    lmList = detector.findPosition(img)

    if lmList and len(lmList) > 20:
        # Cursor follows 9th landmark (Index base)
        cursor = lmList[0]
        index_tip = lmList[8]     # Index tip
        middle_tip = lmList[12]   # Middle tip
        ring_tip = lmList[16]     # Ring tip

        # Apply padding and map to screen size
        raw_x = ((cursor[1] - padding) / (frame_w - 2 * padding)) * screen_w
        raw_y = ((cursor[2] - padding) / (frame_h - 2 * padding)) * screen_h

        # Ensure cursor stays within screen bounds
        raw_x = max(0, min(screen_w, raw_x))
        raw_y = max(0, min(screen_h, raw_y))

        # Smooth cursor movement
        cursor_x = int(prev_x * (1 - smooth_factor) + raw_x * smooth_factor)
        cursor_y = int(prev_y * (1 - smooth_factor) + raw_y * smooth_factor)

        prev_x, prev_y = cursor_x, cursor_y  # Update previous positions

        # Move cursor
        mouse.position = (cursor_x, cursor_y)

        # ✅ Detect movement relative to 9th landmark
        index_moving = (index_tip[2] - cursor[2]) < -300
        middle_moving = (middle_tip[2] - cursor[2]) < -300
        ring_moving = (ring_tip[2] - cursor[2]) < -250
        # print(ring_tip[2] - cursor[2])

        curr_time = time.time()  # Get current time

        # 1️⃣ **Single Click:** Only index finger moves
        if index_moving and not middle_moving and not ring_moving:
            if curr_time - last_click_time > click_cooldown:
                mouse.click(Button.left, 1)
                last_click_time = curr_time

        # 2️⃣ **Double Click:** Index & middle move together
        elif index_moving and middle_moving and not ring_moving:
            if curr_time - last_click_time > click_cooldown:
                mouse.click(Button.left, 2)
                last_click_time = curr_time

        # 3️⃣ **Drag Start:** Index, middle, and ring move together
        elif index_moving and middle_moving and ring_moving and not dragging:
            if curr_time - last_drag_time > drag_cooldown:
                mouse.press(Button.left)
                dragging = True
                last_drag_time = curr_time  # Update last drag time

        # 4️⃣ **Drop (Release Drag):** Open palm (All fingers up)
        fingers = detector.getFingers(img)
        if sum(fingers) == 5 and dragging:
            mouse.release(Button.left)
            dragging = False  # Reset dragging so we can drag again

    # Show Webcam Feed
    cv2.imshow("Webcam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

I0000 00:00:1742726809.455427 7346222 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1742726809.496594 7377867 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742726809.506403 7377867 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


-1